# Make DESI photo-z tables for VO

I was originally going to make a smaller VO table with only:
    
- field
- desi_id
- ra
- dec
- hp_idx
- redshift
- redshift_err
- zspec
 

The HELP field and hp_idx are made in this notebook.

We later decided it would be advantageous to include all columns.

The file is too big to load it all into data so you must first make a column cut with just RA and DEC and then join in the result using stilts.

In [ ]:
from astropy.table import Table, Column

from herschelhelp.utils import position_to_field
from herschelhelp_internal.utils import coords_to_hpidx

## 1. Make csv file with correct columns

The stilts join assumes ordering maintained so be sure no operations here sort the columns in any way.

In [ ]:
print("stilts tpipe cmd='keepcols \"RA DEC\"' dr6dr7_photoz_csp_totcat.fits omode=out out=dr6dr7_reduced.fits")

In [ ]:
#Load a smaller version with just required fields made using Stilts
#stilts tpipe cmd='keepcols "ID RA DEC photo_z photo_zerr spec_z"' 
#./data/dr6dr7_photoz_csp_totcat.fits omode=out out=./data/dr6dr7_reduced.fits
full_tab='./data/dr6dr7_photoz_csp_totcat.fits'
small_tab='./data/dr6dr7_reduced.fits'
desi = Table.read(small_tab)['RA', 'DEC'] #, 'photo_z', 'photo_zerr', 'spec_z']
#desi = Table.read(full_tab)

In [ ]:
#desi['RA'].name = 'ra'
#desi['DEC'].name = 'dec'
#desi['ID'].name = 'desi_id'
#desi['photo_z'].name = 'redshift'
#desi['photo_zerr'].name = 'redshift_err'
#desi['spec_z'].name = 'zspec'

In [ ]:
desi.add_column(Column(
    data=coords_to_hpidx(desi['RA'], desi['DEC'], order=13),
    name="hp_idx"
))

In [ ]:
#desi.write('./data/desi_for_vo_1.fits', overwrite = True)

In [ ]:
desi.add_column(Column(
    data=position_to_field(desi['RA'], desi['DEC'], dmu_loc='../../'),
    name="field"
))

In [ ]:
#cols = ['field', 'desi_id', 'ra', 'dec', 'hp_idx', 'redshift', 'redshift_err', 'zspec']
#cols = ['field', 'ID', 'RA', 'DEC', 'hp_idx', 'photo_z', 'photo_zerr', 'spec_z']
desi.write('./data/desi_for_vo_sub.fits', overwrite = True)

In [ ]:
print("stilts tpipe cmd='keepcols \"field hp_idx\"' desi_for_vo_2.fits omode=out out=desi_for_vo.fits")
print("stilts tjoin nin=2 in1=dr6dr7_photoz_csp_totcat.fits in2=desi_for_vo_sub.fits out=desi_for_vo.fits")
print("stilts tpipe desi_for_vo.fits omode=out ofmt=csv out=desi_for_vo.csv")